## Datapoints generation Shot

In [34]:
import pandas as pd
file_path = 'datasets/Right-All/Right-All-Corrected-events.csv'
main_json_path = 'datasets/Right-All/complete_video_annotation/video/Right-All-mediapipe.json'
# Load CSV
df = pd.read_csv(file_path)
shot_df = df[df['event_name']=='Shot']

In [35]:
shot_df

,video_name,event_name,start_frame_num,end_frame_num,lstm_category
4,Right-All-Corrected.mp4,Shot,507,552,Shot_Status
10,Right-All-Corrected.mp4,Shot,719,758,Shot_Status
16,Right-All-Corrected.mp4,Shot,861,907,Shot_Status
22,Right-All-Corrected.mp4,Shot,981,1021,Shot_Status
28,Right-All-Corrected.mp4,Shot,1092,1130,Shot_Status
...,...,...,...,...,...
1853,Right-All-Corrected.mp4,Shot,44345,44400,Shot_Status
1859,Right-All-Corrected.mp4,Shot,44589,44635,Shot_Status
1865,Right-All-Corrected.mp4,Shot,44900,44949,Shot_Status
1871,Right-All-Corrected.mp4,Shot,45086,45130,Shot_Status


In [36]:
import pandas as pd


# Sort DataFrame by start_frame_num
df = shot_df.sort_values(by='start_frame_num').reset_index(drop=True)

# Initialize variables
background_events = []
prev_end_frame = 0  # Assuming frames start from 0

# Iterate through the dataframe to find gaps
for index, row in df.iterrows():
    start_frame = row['start_frame_num']
    
    if prev_end_frame < start_frame - 1:
        # If there is a gap between the previous end frame and the current start frame
        background_events.append({
            'video_name': row['video_name'],
            'event_name': 'Background',
            'start_frame_num': prev_end_frame + 1,
            'end_frame_num': start_frame - 1,
            'lstm_category': 'Background_Status'
        })
    
    # Update the previous end frame to the current row's end frame
    prev_end_frame = row['end_frame_num']

# Convert background events list to DataFrame
background_df = pd.DataFrame(background_events)

# Combine the original events with the background events
combined_df = pd.concat([df, background_df]).sort_values(by='start_frame_num').reset_index(drop=True)

# Display the result
print(background_df)


                  video_name  event_name  start_frame_num  end_frame_num  \
0    Right-All-Corrected.mp4  Background                1            506   
1    Right-All-Corrected.mp4  Background              553            718   
2    Right-All-Corrected.mp4  Background              759            860   
3    Right-All-Corrected.mp4  Background              908            980   
4    Right-All-Corrected.mp4  Background             1022           1091   
..                       ...         ...              ...            ...   
309  Right-All-Corrected.mp4  Background            44224          44344   
310  Right-All-Corrected.mp4  Background            44401          44588   
311  Right-All-Corrected.mp4  Background            44636          44899   
312  Right-All-Corrected.mp4  Background            44950          45085   
313  Right-All-Corrected.mp4  Background            45131          45317   

         lstm_category  
0    Background_Status  
1    Background_Status  
2    Backgro

In [37]:
import pandas as pd
import json

# Load JSON file with keypoints
with open(main_json_path, 'r') as f:
    keypoints_data = json.load(f)

# Initialize dictionary to store keypoints for each class
keypoints_by_class = {
    'Shot': [],
    'Background': []
}

# Iterate through the combined DataFrame
for _, row in combined_df.iterrows():
    start_frame = row['start_frame_num']
    end_frame = row['end_frame_num']
    event_name = row['event_name']
    
    # Collect keypoints for each frame in the range
    for frame in range(start_frame, end_frame + 1):
        if str(frame) in keypoints_data:
            keypoints_by_class[event_name].append({
                'frame': frame,
                'keypoints': keypoints_data[str(frame)]['keypoints']
            })

# Convert the keypoints dictionary to a DataFrame for easier viewing
shot_keypoints_df = pd.DataFrame(keypoints_by_class['Shot'])
background_keypoints_df = pd.DataFrame(keypoints_by_class['Background'])

# Display the results
print("Shot Keypoints:")
print(shot_keypoints_df)

print("\nBackground Keypoints:")
print(background_keypoints_df)


Shot Keypoints:
       frame                                          keypoints
0        507  [[671, 450], [663, 442], [661, 442], [659, 442...
1        508  [[671, 449], [664, 442], [662, 442], [661, 442...
2        509  [[676, 448], [666, 441], [664, 441], [662, 441...
3        510  [[680, 449], [669, 441], [667, 441], [665, 441...
4        511  [[692, 449], [681, 441], [679, 441], [677, 441...
...      ...                                                ...
13535  45363  [[1480, 344], [1471, 333], [1471, 333], [1471,...
13536  45364  [[1481, 343], [1472, 333], [1472, 333], [1471,...
13537  45365  [[1479, 342], [1469, 333], [1468, 333], [1467,...
13538  45366  [[1479, 339], [1470, 331], [1469, 330], [1467,...
13539  45367  [[1479, 339], [1470, 331], [1469, 330], [1467,...

[13540 rows x 2 columns]

Background Keypoints:
       frame                                          keypoints
0        439  [[655, 473], [649, 464], [647, 463], [646, 463...
1        440  [[662, 470], [656, 461], 

In [38]:
len(shot_keypoints_df)

13540

In [39]:
len(background_keypoints_df)

27606

In [40]:
import os
import numpy as np
import random

# Define the FrameGrouper class
class FrameGrouper:
    def __init__(self, segment_length=20):
        self.segment_length = segment_length

    def group_frames(self, data):
        grouped_values = []
        current_segment = []
        start_frame = 0
        end_frame = max(int(key) for key in data.keys())

        for frame in range(start_frame, end_frame + 1):
            if str(frame) in data:
                current_segment.append(data[str(frame)]['keypoints'])

            if len(current_segment) == self.segment_length:
                grouped_values.append(np.array(current_segment))
                current_segment = []

        # Handle padding if the last group is not full
        if len(current_segment) > 0:
            while len(current_segment) < self.segment_length:
                random_index = random.randint(0, len(current_segment) - 1)
                current_segment.insert(random_index + 1, current_segment[random_index])

            grouped_values.append(np.array(current_segment))

        return grouped_values

def save_structured_data(data_point, label, file_path):
    # Define a structured dtype
    dt = np.dtype([('data', np.float64, data_point.shape), ('label', np.int32)])
    structured_array = np.array([(data_point, label)], dtype=dt)
    np.save(file_path, structured_array)




In [41]:
# Initialize FrameGrouper
frame_grouper = FrameGrouper(segment_length=20)
save_dir_base = 'datasets/Right-All/Shot_datapoints'
position_name = 'Right-All'
# Assume keypoints_by_class contains the grouped keypoints for "Shot" and "Background"
for class_name, keypoints in keypoints_by_class.items():
    grouped_keypoints = frame_grouper.group_frames({str(item['frame']): {'keypoints': item['keypoints']} for item in keypoints})
    
    # Define the directory to save the data
    save_dir = os.path.join(save_dir_base, class_name)
    os.makedirs(save_dir, exist_ok=True)
    
    # Save each segment
    for i, segment in enumerate(grouped_keypoints):
        file_path = os.path.join(save_dir, f'{position_name}_{class_name}_segment_{i}.npy')
        save_structured_data(segment, label=0 if class_name == 'Background' else 1, file_path=file_path)

    print(f'Saved {len(grouped_keypoints)} segments for class {class_name} in {save_dir}')

Saved 677 segments for class Shot in datasets/Right-All/Shot_datapoints/Shot
Saved 1381 segments for class Background in datasets/Right-All/Shot_datapoints/Background


In [11]:
import numpy as np

# Specify the path to your .npy file
file_path = 'datasets/Front-All/Shot_datapoints/Shot/Front-All_Shot_segment_0.npy'

# Load the data
data = np.load(file_path, allow_pickle=True)[0]

# Print the data
print(data['label'])

1


## Datasplit

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Specify the root directory where the subdirectories for each class are located
data_root = 'combined_datapoints_shot'

# Define the directories to store training and validation files
train_dir = 'data_points_splitted/train'
val_dir = 'data_points_splitted/val'


# Create base directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Define the split ratio for training and validation
train_ratio = 0.8  # 80% for training, 20% for validation

# Loop through each class directory
for class_name in os.listdir(data_root):
    class_dir = os.path.join(data_root, class_name)
    if os.path.isdir(class_dir):
        # List all .npy files in the class directory
        files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.endswith('.npy')]
        
        # Split the files into training and validation sets
        train_files, val_files = train_test_split(files, train_size=train_ratio, random_state=42)
        
        # Ensure class subdirectories are created in train and validation directories
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        
        # Copy files to the respective training and validation class directories
        for f in train_files:
            shutil.copy2(f, train_class_dir)
        for f in val_files:
            shutil.copy2(f, val_class_dir)

# Print confirmation
print("Files have been copied to the respective train and validation directories with class structure preserved.")

Files have been copied to the respective train and validation directories with class structure preserved.


## Dataloader

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

class NPYDataset(Dataset):
    def __init__(self, main_directory):
        super().__init__()
        self.main_directory = main_directory
        self.filenames = self._load_filenames()

    def _load_filenames(self):
        # Traverse subdirectories to find all .npy files
        filenames = []
        for root, _, files in os.walk(self.main_directory):
            for file in files:
                if file.endswith('.npy'):
                    full_path = os.path.join(root, file)
                    filenames.append(full_path)
        return filenames

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        file_path = self.filenames[idx]
        data = np.load(file_path, allow_pickle=True)
        keypoints = np.ascontiguousarray(data['data'], dtype=np.float32)
        label = np.float32(data['label'])
        # Convert to tensors
        keypoints = torch.tensor(keypoints)
        label = torch.tensor(label)
        return keypoints, label


In [5]:


# DataLoader setup
data_directory = 'datasets/Front-All/test_data'
dataset = NPYDataset(data_directory)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# val_dataset = NPYDataset('datasets/Front-All/data_points_splitted/val')
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [6]:
len(dataloader)

2040

In [8]:
for keypoints, labels in dataloader:
    print(keypoints.shape, labels)

torch.Size([1, 1, 20, 33, 2]) tensor([[1.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[1.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[1.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[1.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 20, 33, 2]) tensor([[1.]])
torch.Size